# Fuzzy string matching Latin personal names

This notebook is intended as an explanation and illustration of the use of the [RapidFuzz library](https://maxbachmann.github.io/RapidFuzz/index.html) for matching Latin personal names in texts to potential surface forms in a name dictionary. The name dictionary contains name + potential candidate ID's, as is illustrated in the table below.
|Key (name) |Value (mapping entity) |
--- | --- |
|Iulius|Iulius 1 (RE-Iulius_1)|
| | ... |
| | Iulius 16 (RE-Iulius_16) etc.|
|C. Iulius|Iulius 16 (RE-Iulius_16)|
|Gaius Iulius|Iulius 16 (RE-Iulius_16)|

The aim is to match any surface form in text to all its potential IDs in _Paulys Realencyclopädie der classischen Altertumswissenschaft_ (_RE_). The section **Employing RapidFuzz to match** contains the exact code used to match the entities from the gold standard, including a detailed explanation of the multi-token matching.

## Installing and importing Rapidfuzz

In [ ]:
#if Rapidfuzz is not installed, it will be installed:
import pkg_resources

REQUIRED_PACKAGES = [
    'rapidfuzz' 
]

for package in REQUIRED_PACKAGES:
    try:
        dist = pkg_resources.get_distribution(package)
        print('{} ({}) is installed'.format(dist.key, dist.version))
    except pkg_resources.DistributionNotFound:
        print('{} is NOT installed'.format(package))
        !pip install {package}

#import relevant functions

from rapidfuzz import process, fuzz, utils
from rapidfuzz.process import extractOne, extract
from rapidfuzz.fuzz import ratio, token_ratio
from rapidfuzz.distance import Indel, Levenshtein
import pandas as pd
import os
import json

## Testing performance of different types of matching
Many different options exist for Fuzzy matching based on the type of distance used and the threshold employed.

First, a comparison between partial and normal ratio is offered to demonstrate the complications when using partial matching. Second, two different type of distance measures are compared Indel (standard in RapidFuzz) and Levenshtein (also a common type of distance). Other distances such as Jaro-Winkler are not considered at this moment but could be employed if desired.

### Partial vs. Normal ratio

The surface form **seruius galba** should match with the potential surface form in the name dictionary (e.g. **Seruius Sulpicius Galba**). Using partial_token_ratio will match 100 with any token that contains the substring and will thus match a 100 percent with any name that contains either seruius or galba. The same is true for the token_ratio function. This is shown below. An issue is encountered when a longer string contains a complete other string as substring such as is the case with **Uespasianus** and **Asia**.  

In [ ]:
# Lowercase the strings
str1 = "seruius galba".lower()
str2 = "Seruius Sulpicius Galba".lower()
str3= "Seruius".lower()
str4 = "Galba".lower()
str5 = "asia".lower()
str6 = "Uespasianus".lower()

# Calculate the partial ratio
score = fuzz.partial_token_ratio(str1, str3)

print(f'We want the surface form "seruius galba" to match to (1) "Seruius Sulpicius Galba", (2) "Seruius" and (3) "Galba". \nPartial_token_ratio gives \n {fuzz.partial_token_ratio(str1, str2)} for (1) \n {fuzz.partial_token_ratio(str1, str3)} for (2) \n {fuzz.partial_token_ratio(str1, str4)} for (3) \n\nToken_ratio gives \n {fuzz.token_ratio(str1, str2)} for (1) \n {fuzz.token_ratio(str1, str3)} for (2) \n {fuzz.token_ratio(str1, str4)} for (3) \n\nWhen matching the string "uespasianus", the issue of partial matching becomes clear \n Partial score uespasianus - Asia: {fuzz.partial_token_ratio(str5, str6)} \n Normal ratio score uespasianus - Asia: {fuzz.token_ratio(str5, str6)}')

Partial matching will thus not suffice for matching as it will cause noise in the result. Therefore, the normal Token_ratio is preferred. 

### Indel or Levenshtein distance
The main difference between these two distances is their approach to substitution. Compare what happens in cases of insertion and substitution:

In [ ]:
str1 = "Saeuinus".lower() #insertion
str2 = "Saeuinius".lower()
str3 = "Uologaesus".lower() #substitution
str4 = "Uologaeses".lower()

print(f'The standard RapidFuzz token_ratio relies upon Indel distance and gives the following scores for insertion (+1) and substitution (e ipv u): \n Insertion: {fuzz.token_ratio(str1, str2) } \n Substitution {fuzz.token_ratio(str3, str4)} \n\nCompare this to the Levenshtein similarity: \n Insertion: {Levenshtein.normalized_similarity(str1, str2)} \n Substitution {Levenshtein.normalized_similarity(str3, str4)}')

This shows that for Indel distance, substitution is considered a more complicated change than insertion whereas Levenshtein distance is the opposite, which is explained in the following:

In [ ]:
#Indel explanation of getting from str1 to 2 and from 3 to 4
print(f'Indel distance between the strings is calculated based on the following information: \n Insertion: {Indel.editops(str1, str2)} \n Substitution: {Indel.editops(str3, str4)}\n')
print(f'Levenshtein distance between the strings is calculated based on the following information: \n Insertion: {Levenshtein.editops(str1, str2)} \n Substitution: {Levenshtein.editops(str3, str4)}')

This shows that the substitution in Indel distance actually exists of two edits, an insert and a deletion, whereas for Levenshtein it is simply one edit, replace. Both functions are normalized, which mean they also take into account the length of the string (score = distance / (len1 + len2)). For our article, we chose to work with Indel distance as it is the standard in the Rapidfuzz library.

## Rapidfuzz Extract
The extract process provided by RapidFuzz allows for matching a string to a list of other strings, extracting each potential match. It allows you to specify the following parameters: 
- search string (query): in our case the **lemma** offered for a token or the **combination of lemmas** for multi-token entities (when lemma is not available, token is used)
- choices: the list of names to match the search string to, which is provided by the name dictionary keys
- scorer: define a scorer either provided by RapidFuzz or self-created for edit distance; we selected token_ratio
- processor: we used the "default_process" trims whitespaces, ignores numbers and lowercases the strings
- score_cutoff: any matches with a score below this score will not be returned; we selected a cutoff of 88
- score_hint: optional argument for an expected score to be passed to the scorer; no expected score is specified
- limit: maximum number of candidates to be generated (The list is sorted by the similarity. When multiple choices have the same similarity, they are sorted by their index); no limit is specified

### Cutoff explained
The cutoff score was determined based on extensive experimentation, finding a balance between including too much noise and still matching often recurring differences. Two often occurring differences are:
- No lemma is provided for the token and the thus the token itself is used for matching. These are often in a different case than the surface forms recorded in the name dictionary. For example, the name dictionary does not contain the forms _Hannibaliano_, just _Hannibalianus_. It is important that these still match if that is the only difference between the tokens. The example below illustrates that placing the cutoff at 88 will allow longer strings to be matched in this way, but shorter strings do not. However, placing the cutoff lower than 88, will introduce more noise in the data.
- The lemma provided in the text has a different ending than the _RE_ entry: for example, the lemma recorded for _Laco_ in the name _Cornelius Laco_ is _Lacon_. The example below illustrates that placing the cutoff at 88 will allow this type of difference still to be matched

In [ ]:
str1 = "Hannibaliano".lower() #insertion + deletion long string
str2 = "Hannibalianus".lower()

str3 = "Iulio".lower() #insertion + deletion short string
str4 = "Iulius".lower()

str5 = "Lacon".lower() #deletion + short string
str6 = "Laco".lower()

print(f'The score for matching a token without lemma (cased) scores as follows: \n Long string: {fuzz.token_ratio(str1, str2)} \n Short string: {fuzz.token_ratio(str3, str4)} \n\nThe score for matching a lemma with a different ending than the Latinized spelling in the RE: \n Score: {fuzz.token_ratio(str5, str6)}')

# Employing RapidFuzz to match
The following code can be used to match any dictionary with token_ids as keys and names as values to the name dictionary that is available on the GitHub. In addition, the code used to exact match multi-token entities, i.e. when all sub-parts of the multi-token exist for the predicted IDs. For this, the ID dictionary is used that contains for each _RE_-ID all potential surface forms. For multi-token entities, the token_id used in the dictionary, is the id of the first token of the entity (the B- labelled entity).

Below the process is illustrated with the sample sentence from Tacitus _Historiae_ 1 (sent_id = TacHist1-Q-05-26):
>_tardum Galbae iter et cruentum interfectis Cingonio Uarrone consule designato et Petronio Turpiliano consulari ille ut Nymphidii socius hic ut dux Neronis inauditi atque indefensi tamquam innocentes perierant_


In [ ]:
#import name dictionary
with open('./Real_name_dict_Latin.json', 'r') as json_file:
    name_dict = json.load(json_file)   

#import id dictionary
with open('./Real_id_dict_Latin.json', 'r') as json_file:
    id_dict = json.load(json_file)

In [ ]:
#establish a dictionary with token_ids as key and the strings to match as value
test_dictionary = {
    "TokenURI=http://lila-erc.eu/data/corpora/Lasla/id/corpus/TacitusTac%20Historiae/Tacitus_TacHistoriae_TacHist1.BPN_t_0000689": "galba",
    "TokenURI=http://lila-erc.eu/data/corpora/Lasla/id/corpus/TacitusTac%20Historiae/Tacitus_TacHistoriae_TacHist1.BPN_t_0000694": "cingonius uarro", #multi-token entity uses the token_id of the first token as key
    "TokenURI=http://lila-erc.eu/data/corpora/Lasla/id/corpus/TacitusTac%20Historiae/Tacitus_TacHistoriae_TacHist1.BPN_t_0000699": "petronius turpilianus",
    "TokenURI=http://lila-erc.eu/data/corpora/Lasla/id/corpus/TacitusTac%20Historiae/Tacitus_TacHistoriae_TacHist1.BPN_t_0000704": "nymphidius",
    "TokenURI=http://lila-erc.eu/data/corpora/Lasla/id/corpus/TacitusTac%20Historiae/Tacitus_TacHistoriae_TacHist1.BPN_t_0000709": "nero"   
}

In [ ]:
# Create a new dictionary to store the results
matched_dict = {}

# Iterate over each item in the original dictionary 
for key, value in test_dictionary.items():
    #use RapidFuzz extract with token_ratio to match each surface form to all potential matches in the name dictionary that have a score higher than 88
    results = extract(value, name_dict.keys(), scorer=token_ratio, processor=utils.default_process, score_cutoff = 88, limit = None) 

    #empty list and set to store the results
    match_lemmas = []
    ids = set()
    
    for lemma, score, ind in results:
        #for each result, store the matched form in a list
        match_lemma = lemma, score
        match_lemmas.append(match_lemma)
        #retreive all potential RE_ids from the name dictionary and store in a set to ensure unique values only (i.e. remove all duplicate predictions)
        ids.update(name_dict[lemma])
    #store the results in a dictionary with the token_id as key and a list containing the string to match, the potential surface form matches, and the unique list of potential ids
    matched_dict[key] = [value, match_lemmas, ids]    

#for further processing & printing store the token_id and the potential matching RE_ids in a dictionary
simple_match_fuzz = {key: value[2] for key, value in matched_dict.items()}
print(simple_match_fuzz)

## Multi-token entities
The code below uses the predictions from the previous block and limits the predictions for multi-token entities to only those IDs that have an exact match as potential surface form. 

In [ ]:
#provide for multi-tokens only those ids that contain all sub-parts of the multi-token in a surface form
matched_dict_multi = {}

for key, value in matched_dict.items():
    #iterate over dictionary containing token_id as key and a list containing the string to match, the potential surface form matches, and the unique list of potential ids and retrieve those containing more than one token
    nam_parts = value[0].split()
    if len(nam_parts) > 1:
        pot_match = []
        for re_id in value[2]:
            #for each RE_id predicted in the previous step, check if the lower case of all parts of the multi-token (nam_parts) matches exact any of the potential surface forms (nam_comps)
            nam_comps = id_dict[re_id]
            if set([item.lower() for item in nam_parts]).issubset(set([item.lower() for item in nam_comps])):
                  pot_match.append(re_id)
        
        #if such a match is found, store this new prediction in the new dictionary, if not, retain the original predictions
        if len(set(pot_match)) != 0:
            matched_dict_multi[key] = value[0], value[1], set(pot_match)
        else:
            matched_dict_multi[key] = value
    #if the entity is not longer than one word, retain the original prediction
    else:
        matched_dict_multi[key] = value
        
#for further processing & printing store the token_id and the potential matching RE_ids in a dictionary
simple_match_multi = {key: list(value[2]) for key, value in matched_dict_multi.items()}
print(simple_match_multi)

In [ ]:
for key, value in matched_dict_multi.items():
    org_length = len(simple_match_fuzz[key])
    multi_length = len(simple_match_multi[key])
    entity = value[0]
    print(f"The number of predictions for entity '%s': \norignal: %d \t multi-token: %d \n" %(entity, org_length, multi_length))

The case of _cingonius uarro_ can illustrate what happens. For the original predictions, the potential surface forms are retrieved and checked for an exact match in any of the potential surface forms. As you can see in the printed results of the code below, only **RE-Cingonius** has a surface form that contains an exact match for both _cingonius_ and _uarro_.

In [ ]:
org_preds = matched_dict['TokenURI=http://lila-erc.eu/data/corpora/Lasla/id/corpus/TacitusTac%20Historiae/Tacitus_TacHistoriae_TacHist1.BPN_t_0000694'][2]
nam_parts = matched_dict['TokenURI=http://lila-erc.eu/data/corpora/Lasla/id/corpus/TacitusTac%20Historiae/Tacitus_TacHistoriae_TacHist1.BPN_t_0000694'][0].split()

for re_id in org_preds:
    nam_comps = id_dict[re_id]
    print(f"For %s potential surface forms are %s"%(re_id, nam_comps))
    if set([item.lower() for item in nam_parts]).issubset(set([item.lower() for item in nam_comps])):
        print(f"MATCH: The sub-parts all (%s) have an exact match in a surface form of %s\n"%(nam_parts, re_id))
    else:
        print(f"At least one of the sub-parts (%s) does not have an exact match in a surface form of %s\n"%(nam_parts, re_id))
    